In [1]:
pip install pyspark

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 54.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=d128cbd6dfc798d77d20c08404028b90eca59fad34370e65b28d8a3adf59d442
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [33]:
#importing pyspark library
import pyspark as ps

In [34]:
#setting configaration and creating sql context object 
conf = ps.SparkConf().setAll([('spark.executor.memory', '16g'), ('spark.driver.memory', '16g')])
sc = ps.SparkContext(conf=conf)
from pyspark.sql import SQLContext

sql_context = SQLContext(sc)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [35]:
#connectiong to drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [36]:
#setting data set path
file="/content/gdrive/MyDrive/livetweets.csv"

In [37]:
#reading csv with headers
tweets = sql_context.read.format('com.databricks.spark.csv').option('header','true').option("delimiter", ",").load(file)

In [38]:
#tweets count
print(tweets.count())

305750


In [39]:
#display complete tweet first 100
tweets.show(100,truncate=False)


+------------------------------------------------------------------------------------------------------------------------------------------------------------+----+
|Text                                                                                                                                                        |_c1 |
+------------------------------------------------------------------------------------------------------------------------------------------------------------+----+
|RT @Mii_Mario_: The kitty cat sleeping with waluigi https://t.co/tMcEt9N7B3                                                                                 |null|
|RT @DogsHistorian: From Pydna - Funerary Stele Depicting a Young guy with his Dog - IV century BC - Archaeological #museum of Dion Macedoni…                |null|
|RT @RoninDudeArt: Sassy misunderstood and brought a squirtgun to a paintball match 😅 https://t.co/6HM5AS5BxG                                               |null|
|gonna try this m

In [40]:
tweets=tweets.drop("_c1")

In [41]:
#logic to remove RT
tweets.createOrReplaceTempView("tweets")

In [42]:
tweetsWithOutRT = sql_context.sql("SELECT * FROM tweets where text NOT LIKE '%RT%'")

In [43]:
#tweetsWithOutRT=tweetsWithOutRT.drop("_c1")

In [44]:
tweetsWithOutRT.show()

+--------------------+
|                Text|
+--------------------+
|gonna try this mi...|
|The older we get ...|
|Lawyer for journa...|
|     THIS IS SO CUTE|
|Yeah what happene...|
|@BarbaraDixon__ G...|
|Just sold! Get yo...|
|US health officia...|
|— 1990 🤍 with hi...|
|Not sure this is ...|
|Feels like we wen...|
|            Twittodz|
|My dog and I look...|
|Blew this into sh...|
|@GravisMushnick T...|
|@NotsNfts  So exc...|
|HiMERU looks at U...|
|I̵t̵'̵s̵ ̵b̵e̵e̵n...|
|Samsung Galaxy S2...|
|This man will joi...|
+--------------------+
only showing top 20 rows



In [45]:
print(tweetsWithOutRT.count())

71876


In [46]:
#preprocesing text
from pyspark.sql.functions import regexp_replace

In [47]:
#removing user names
tweetsWithOutRT = tweetsWithOutRT.withColumn("text",  regexp_replace("text", "@[A-Za-z0-9]+", ""))
 

In [48]:
import pyspark.sql.functions as f

In [49]:
#removing web links and special  characters
tweetsWithOutRT = tweetsWithOutRT.withColumn("text", regexp_replace("text", "(?:\@|http?\://|https?\://|www)\S+", " "))
#tw = tw.withColumn("Tweet", regexp_replace("Tweet", "#[A-Za-z0-9]+", ""))
tweetsWithOutRT = tweetsWithOutRT.withColumn("text", regexp_replace("text", "'", " "))
tweetsWithOutRT = tweetsWithOutRT.withColumn("text", regexp_replace("text", "[^A-Za-z0-9]+", " "))
tweetsWithOutRT = tweetsWithOutRT.withColumn("text", f.lower(f.col("text")) )
tweetsWithOutRT = tweetsWithOutRT.na.drop()

In [50]:
tweetsWithOutRT = tweetsWithOutRT.withColumn("text",regexp_replace("text", "[\\r\\n\\t]", " ") )

In [51]:
tweetsWithOutRT.show()

+--------------------+
|                text|
+--------------------+
|gonna try this mi...|
|the older we get ...|
|lawyer for journa...|
|     this is so cute|
|yeah what happene...|
|     good for you x |
|just sold get you...|
|us health officia...|
|      1990 with him |
|not sure this is ...|
|feels like we wen...|
|            twittodz|
|my dog and i look...|
|blew this into sh...|
| these communists...|
| so excited to ge...|
|  himeru looks at u |
|i t s b e e n a q...|
|samsung galaxy s2...|
|this man will joi...|
+--------------------+
only showing top 20 rows



In [52]:
#removing null tweets
tweetsWithOutRT = tweetsWithOutRT.where(tweetsWithOutRT.text!='null')

In [53]:
print(tweetsWithOutRT.count())

71876


In [54]:
#tweetspreprocessed.to_csv('tweetspreprocessed.csv', encoding='utf-8')
#saving csv
tweetsWithOutRT.write.csv("/content/gdrive/MyDrive/Colab Notebooks/preprocesedlivetweets")

In [55]:
#killing context object
sc.stop()